# 前処理

In [62]:
import pandas as pd
import matplotlib

ModuleNotFoundError: No module named 'matplotlib'

In [51]:
%reload_kedro

2021-04-26 09:52:26,004 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-04-26 09:52:26,061 - root - INFO - ** Kedro project project_student_cup_2021_spr
2021-04-26 09:52:26,062 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-04-26 09:52:26,067 - root - INFO - Registered line magic `run_viz`


## データ読み込み

In [52]:
train_df = catalog.load('train')
test_df = catalog.load('test')

2021-04-26 09:52:26,075 - kedro.io.data_catalog - INFO - Loading data from `train` (CSVDataSet)...
2021-04-26 09:52:26,088 - kedro.io.data_catalog - INFO - Loading data from `test` (CSVDataSet)...


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## trainとtestを結合

In [53]:
def merge_train_test(train_df: pd.DataFrame, test_df: pd.DataFrame):
    train_df['is_train'] = 1
    test_df['is_train'] = 0
    res = pd.concat([train_df, test_df])
    res.reset_index(inplace=True, drop=True)
    return res

In [54]:
def split_train_test(df: pd.DataFrame, label:str = 'is_train'):
    df_train = df[df[label] == 1]
    df_test = df[df[label] == 0]
    df_train.reset_index(inplace=True, drop=True)
    df_test.reset_index(inplace=True, drop=True)
    return df_train, df_test

In [55]:
res = merge_train_test(train_df, test_df)

In [56]:
res.tempo.unique()

array(['121-152', '153-176', '64-76', '177-192', '97-120', '77-96',
       '193-208', '57-63', '209-220', '51-56', '0-40', '41-50'],
      dtype=object)

In [57]:
res

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,is_train
0,0,10.0,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H,1
1,1,8.0,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I,1
2,2,3.0,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E,1
3,3,10.0,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C,1
4,4,3.0,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,34,366668,0.117208,0.316026,0.502289,-2.909329,0.824708,0.320241,0.206592,0.198859,121-152,region_B,0
8088,8088,NaN,53,240081,0.225540,0.391226,0.483655,-5.013634,0.630808,0.163884,0.205218,0.179438,121-152,region_I,0
8089,8089,NaN,48,217839,0.215571,0.828567,0.689747,-8.974036,0.697981,0.318250,0.123748,0.135281,121-152,region_H,0
8090,8090,NaN,45,249612,0.205074,0.730554,0.580581,-1.915320,0.828423,0.360663,0.165837,0.162821,121-152,region_B,0


In [59]:
def extract_tempo_data(df):
    temp = df['tempo'].str.split('-', expand=True)
    temp.columns = ['tempo_low', 'tempo_high']
    temp['tempo_low'] = temp['tempo_low'].astype('int')
    temp['tempo_high'] = temp['tempo_high'].astype('int')
    df = pd.concat([df, temp], axis=1)
    df['tempo_mean'] = (df['tempo_low'] + df['tempo_high']) / 2
    df['tempo_diff'] = df['tempo_high'] - df['tempo_low'] 
    return df

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,is_train,tempo_low,tempo_high,tempo_mean,tempo_diff
0,0,10.0,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H,1,121,152,136.5,31
1,1,8.0,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I,1,153,176,164.5,23
2,2,3.0,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E,1,64,76,70.0,12
3,3,10.0,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C,1,177,192,184.5,15
4,4,3.0,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown,1,97,120,108.5,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,34,366668,0.117208,0.316026,0.502289,-2.909329,0.824708,0.320241,0.206592,0.198859,121-152,region_B,0,121,152,136.5,31
8088,8088,NaN,53,240081,0.225540,0.391226,0.483655,-5.013634,0.630808,0.163884,0.205218,0.179438,121-152,region_I,0,121,152,136.5,31
8089,8089,NaN,48,217839,0.215571,0.828567,0.689747,-8.974036,0.697981,0.318250,0.123748,0.135281,121-152,region_H,0,121,152,136.5,31
8090,8090,NaN,45,249612,0.205074,0.730554,0.580581,-1.915320,0.828423,0.360663,0.165837,0.162821,121-152,region_B,0,121,152,136.5,31


In [60]:
extract_tempo_data(res)

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,is_train,tempo_low,tempo_high,tempo_mean,tempo_diff
0,0,10.0,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H,1,121,152,136.5,31
1,1,8.0,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I,1,153,176,164.5,23
2,2,3.0,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E,1,64,76,70.0,12
3,3,10.0,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C,1,177,192,184.5,15
4,4,3.0,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown,1,97,120,108.5,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087,8087,NaN,34,366668,0.117208,0.316026,0.502289,-2.909329,0.824708,0.320241,0.206592,0.198859,121-152,region_B,0,121,152,136.5,31
8088,8088,NaN,53,240081,0.225540,0.391226,0.483655,-5.013634,0.630808,0.163884,0.205218,0.179438,121-152,region_I,0,121,152,136.5,31
8089,8089,NaN,48,217839,0.215571,0.828567,0.689747,-8.974036,0.697981,0.318250,0.123748,0.135281,121-152,region_H,0,121,152,136.5,31
8090,8090,NaN,45,249612,0.205074,0.730554,0.580581,-1.915320,0.828423,0.360663,0.165837,0.162821,121-152,region_B,0,121,152,136.5,31


In [61]:
res['popularity'].hist()

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [29]:
def one_hot_sklearn(df: pd.DataFrame, drop='first', na_as_category=True) -> pd.DataFrame:
    """
    Notes
    -----
    int型も処理される。必要な列だけ処理されるように注意する。
    """
    if na_as_category:
        df = df.fillna("NA")
        ohe = OneHotEncoder(sparse=False, drop=drop)
        encoded = ohe.fit_transform(df.values)
        label = ohe.get_feature_names(df.columns)
        res = pd.DataFrame(encoded, columns=label, dtype=np.int8)
    else:
        res = pd.DataFrame(index=df.index)
        cat_cols = df.columns
        for c in cat_cols:
            not_null = df[df[c].notnull()][[c]]
            ohe = OneHotEncoder(sparse=False, drop=drop)
            encoded = ohe.fit_transform(not_null.values)
            label = ohe.get_feature_names(not_null.columns)
            res = pd.merge(res, pd.DataFrame(encoded, columns=label, dtype=np.int8, index=not_null.index),
                           left_index=True, right_index=True, how='left')

    return res

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
train

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,is_train
0,0,10.0,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H,1
1,1,8.0,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I,1
2,2,3.0,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E,1
3,3,10.0,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C,1
4,4,3.0,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,4041,10.0,38,246309,0.128795,0.329718,0.575830,-5.252543,0.509531,0.193781,0.187331,0.158197,121-152,region_P,1
4042,4042,5.0,23,208734,0.778732,0.228757,0.394283,-14.225700,0.322871,0.253108,0.141218,0.295608,121-152,region_D,1
4043,4043,10.0,30,407016,0.402050,0.462374,0.672265,-10.711253,0.646571,0.228189,0.152270,0.163483,97-120,region_E,1
4044,4044,10.0,25,204957,0.208096,0.465511,0.523514,-7.855946,0.508910,0.247820,0.202238,0.210184,77-96,region_R,1


In [8]:
train_df

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region
0,0,10,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H
1,1,8,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I
2,2,3,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E
3,3,10,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C
4,4,3,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,4041,10,38,246309,0.128795,0.329718,0.575830,-5.252543,0.509531,0.193781,0.187331,0.158197,121-152,region_P
4042,4042,5,23,208734,0.778732,0.228757,0.394283,-14.225700,0.322871,0.253108,0.141218,0.295608,121-152,region_D
4043,4043,10,30,407016,0.402050,0.462374,0.672265,-10.711253,0.646571,0.228189,0.152270,0.163483,97-120,region_E
4044,4044,10,25,204957,0.208096,0.465511,0.523514,-7.855946,0.508910,0.247820,0.202238,0.210184,77-96,region_R
